In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
from wordcloud import WordCloud
import seaborn as sns

In [ ]:
url = 'https://www.iefimerida.gr/politiki/ano-kato-i-elliniki-lysi-o-belopoylos-fobithike-apostasia-kai-kalese-se-apologia-boyleytes'
response = requests.get(url)


if response.status_code == 200:

    soup = BeautifulSoup(response.content, 'html.parser')


    title = soup.find("h1", {"class": "f-big w-bold"})


    if title:
        print("Title:", title.get_text(strip=True))
    else:
        print("Title not found.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [ ]:
url = 'https://www.iefimerida.gr/politiki/ano-kato-i-elliniki-lysi-o-belopoylos-fobithike-apostasia-kai-kalese-se-apologia-boyleytes'


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


response = requests.get(url, headers=headers)


if response.status_code == 200:

    soup = BeautifulSoup(response.content, 'html.parser')


    title = soup.find("h1", {"class": "f-big w-bold"})
    category = soup.find("div", {"class" : "details f-details-alt c-red"})
    author = ("a")

    if title:
        print("Title:", title.get_text(strip=True))
    else:
        print("Title not found.")
    if category:
        print("category:", category.get_text(strip=True))
    else:
        print("category not found.")
    author_div = soup.find("div", {"class": "author f-letter-spacing c-black w-bold f-details-author font-family-main"})
    if author_div:

        author = author_div.find("a")
        if author:
            print("Author:", author.get_text(strip=True))
        else:
            print("Author <a> tag not found.")
    else:
        print("Author div not found.")
    datetime_div = author_div.find("div", {"class": "f-details c-black w-regular details-below"})
    if datetime_div:
       publication_time = datetime_div.find("time")
       if publication_time:
            print("Datetime:", publication_time.get_text(strip=True))
       else:
            print("Datetime not found.")
    else:
        print("datetime div")

    main_content = soup.find("div", {"class": "field--name-body"})


    for child in main_content.children:

        if child.name == 'p':
            print(child.get_text(strip=True))

        elif child.name == 'h2':
            print(child.get_text(strip=True))

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [ ]:
def process_url(url):

    return url
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


response = requests.get(url, headers=headers)


base_url = 'https://www.iefimerida.gr/politiki?page='
urls = []

for page_num in range(1406,1407):
    print(f"Processing page: {page_num}")
    url = base_url + str(page_num)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all("article")
    for article in articles:
      article_url = article.find("a")["href"]

      urls.append(article_url)




In [ ]:
def process_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        article_urls = []

        articles = soup.find_all("article")
        for article in articles:
            article_url = article.find("a")["href"]
            if article_url.startswith('/'):
                article_url = 'https://www.iefimerida.gr' + article_url
            article_urls.append(article_url)

        return article_urls
    else:
        print(f"Failed to retrieve page {url}. Status code: {response.status_code}")
        return []

base_url = 'https://www.iefimerida.gr/politiki?page='
start_page = 1406
end_page = 1407

all_article_urls = []

# Iterate through pages and process each URL
for page_num in range(start_page, end_page + 1):
    print(f"Processing page: {page_num}")
    url = base_url + str(page_num)
    article_urls = process_url(url)
    all_article_urls.extend(article_urls)

# Now all_article_urls contains URLs of articles from all pages
print(f"Total articles found: {len(all_article_urls)}")

# Example: Print the first few article URLs
print("Example article URLs:")
for idx, article_url in enumerate(all_article_urls[:5], start=1):
    print(f"{idx}. {article_url}")
    # You can add code here to process each article URL and extract its content


In [ ]:
urls

In [ ]:
from urllib.parse import urljoin
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def process_url(url):
    # Check if the URL is relative (does not start with 'http' or 'https')
    if not url.startswith('http'):
        # Prepend the base domain
        url = urljoin('https://www.iefimerida.gr', url)
    return url

def get_with_retries(url, headers, max_retries=3):
    session = requests.Session()
    retries = Retry(total=max_retries, backoff_factor=1, status_forcelist=[502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session.get(url, headers=headers)

base_url = 'https://www.iefimerida.gr/politiki?page='
urls = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

for page_num in range(1406, 1525):
    print(f"Processing page: {page_num}")
    url = base_url + str(page_num)
    try:
        response = get_with_retries(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        iefimerida_articles = soup.find_all('article')
        print(f"Found {len(iefimerida_articles)} articles on page {page_num}")

        for i, article in enumerate(iefimerida_articles):
            link_tag = article.find('a')

            if link_tag:
                raw_url = link_tag["href"]
                processed_url = process_url(raw_url)
                urls.append(processed_url)
                print(f"Added URL from article {i+1} on page {page_num}: {processed_url}")
            else:
                print(f"Missing link in article {i+1} on page {page_num}")

        time.sleep(5)

    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve page {page_num}: {e}")

data = {'URL': urls}
print("Scraping completed. Total URLs collected:", len(urls))
print(data)



In [ ]:
iefimerida_urls = pd.DataFrame(data)
iefimerida_urls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
iefimerida_urls = pd.read_csv("/content/drive/MyDrive/iefimerida_urls")

In [ ]:
full_articles_list = []
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

for index, processed_url in enumerate(iefimerida_urls['URL']):
    full_article_dict = {}
    try:
        response = requests.get(processed_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        full_article_dict['site'] = "iefimerida.gr"
        full_article_dict['url'] = processed_url

        title_tag = soup.find("h1", {"class": "f-big w-bold"})
        title = title_tag.get_text(strip=True) if title_tag else 'No Title Found'
        full_article_dict['title'] = title

        category_tag = soup.find("div", {"class": "details f-details-alt c-red"})
        category = category_tag.get_text(strip=True) if category_tag else 'No Category Found'
        full_article_dict['category'] = category

        author_tag = soup.find("a")
        author = author_tag.get_text(strip=True) if author_tag else 'No Author Found'
        full_article_dict['author'] = author

        datetime_div = soup.find("div", {"class": "f-details c-black w-regular details-below"})
        if datetime_div:
            publication_time_tag = datetime_div.find("time")
            publication_time = publication_time_tag.get_text(strip=True) if publication_time_tag else 'No Publication Time Found'
        else:
            publication_time = 'No Publication Time Found'
        full_article_dict['publication_time'] = publication_time

        main_content = soup.find("div", {"class": "field--name-body"})
        content = ''
        if main_content:
            for child in main_content.children:
                if child.name == 'p':
                    content += child.get_text(strip=True) + '\n'
                elif child.name == 'h2':
                    content += '\n' + child.get_text(strip=True) + '\n'
        full_article_dict['content'] = content.strip()

    except Exception as e:
        print(f"Error processing {processed_url}: {e}")
        full_article_dict = {
            'site': "iefimerida.gr",
            'url': processed_url,
            'title': 'Error',
            'category': 'Error',
            'author': 'Error',
            'publication_time': 'Error',
            'content': 'Error'
        }

    print(f"Processed {index + 1}/{len(iefimerida_urls['URL'])}: {processed_url}")

    full_articles_list.append(full_article_dict)

    time.sleep(1.5)

    iefimerida_grel = pd.DataFrame(full_articles_list)


In [ ]:
epidomata-thermansis ilektrokinisi messinis genoktonias genoktonia efiboy-rapti tsoyra-kideia thanasis-tsoyras thanasi-tsoyra fylaki-o-frenti-mpeleris synelifthi-16hronos erntogan-i-kilitsntarogloy-o-k-filis-mila-sto-iefimeridagr maria-ston-ebro ypiresies-gia-toys-asfalismenoys karoloy 15hronis-mathitrias melos-toy-demp brand-name karagiannis-aytokinitodromos elleimma-erga tzoker-tzak-pot-klirosi oikotrofeio-rapti tragodia-beligradi sti-serbia panspoydastikis hurley aystralia diamesolabisi-ellinotoyrkika galloi-syndikalistes toyrismos-2023 poios-eiserhetai gia-tin-ylopoiisi dialogos-i kairos-protomagia toy-soydan harlei horo-e kratikopoiisi-ethnikis syllogo-athinon soydan mykonos zormpa mpermpok anomias boak kriti markopoyloy

In [ ]:
iefimerida_grel.to_csv("/content/drive/MyDrive/iefimerida_grel", index = False)

In [ ]:
iefimerida_grel

In [ ]:

rubbish = [
    "epidomata-thermansis", "ilektrokinisi", "messinis", "genoktonias", "genoktonia",
    "efiboy-rapti", "thanasis-tsoyras", "tsoyra", "fylaki-o-frenti-mpeleri", "synelifthi-16hronos",
    "erntogan-i-kilitsntarogloy-o-k-filis-mila-sto-iefimeridagr", "maria-ston-ebro", "ypiresies-gia-toys-asfalismenoys",
    "karoloy", "15hronis-mathitrias", "melos-toy-demp", "brand-name", "karagiannis-aytokinitodromos",
    "elleimma-erga", "tzoker-tzak-pot-klirosi", "oikotrofeio-rapti", "tragodia-beligradi", "sti-serbia",
    "panspoydastikis", "hurley", "aystralia", "diamesolabisi-ellinotoyrkika", "galloi-syndikalistes",
    "toyrismos-2023", "poios-eiserhetai", "gia-tin-ylopoiisi", "dialogos-i", "kairos-protomagia",
    "harlei", "horo-e", "kratikopoiisi-ethnikis", "syllogo-athinon", "soydan", "mykonos", "zormpa",
    "mpermpok", "anomias", "boak", "kriti", "markopoyloy"
]
pattern = '|'.join(rubbish)

iefimerida_grel = iefimerida_grel[~iefimerida_grel['url'].str.contains(pattern, case=False, na=False)]

iefimerida_grel

In [ ]:
iefimerida_grel.rename(columns={
    'content': 'text',
    'publication_time': 'datetime'
}, inplace=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
iefimerida_grel = pd.read_csv("/content/drive/MyDrive/iefimerida_grel")
iefimerida_grel

In [ ]:
iefimerida_grel = pd.read_csv('C:\\Users\\stath\\iefimerida_greek_elections')

In [ ]:
iefimerida_grel.shape

In [ ]:
iefimerida_grel.to_csv('C:\\Users\\stath\\iefimerida_greek_elections', index = False)

In [ ]:
iefimerida_grel.rename(columns={
    'content': 'text',
    'publication_time': 'datetime'
}, inplace=True)

In [ ]:
iefimerida_grel["text"][55]

In [ ]:
iefimerida_grel["text"] = iefimerida_grel["text"].str.replace("\n", "")
iefimerida_grel["text"] = iefimerida_grel["text"].str.replace(':Φωτογραφίες: Νίκος Μαρουλίδης', "")
iefimerida_grel["text"] = iefimerida_grel["text"].str.replace("Εκλογές 2023 -Εικόνες από το Κέντρο Τύπου στο Ζάππειο Μέγαρο", "")
iefimerida_grel["text"] = iefimerida_grel["text"].str.replace("\n«Ό,τι πει η κάλπη", "«Ό,τι πει η κάλπη")
iefimerida_grel["text"] = iefimerida_grel["text"].str.replace("\nΔείτε", "Δείτε")

In [ ]:
iefimerida_grel = iefimerida_grel.drop(columns=['author'])

In [ ]:
pd.set_option('display.max_rows', 1190)

In [ ]:
iefimerida_grel['datetime'] = pd.to_datetime(iefimerida_grel['datetime'], format='%d/%m/%Y %H:%M', errors='coerce')


In [ ]:
iefimerida_grel


In [ ]:
short_df = iefimerida_grel[["title", "datetime"]]

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
iefimerida_grel['κόμματα'] = ''

In [ ]:
iefimerida_grel = iefimerida_grel.drop([40,41,86,92,100,121,122,138,141,149,162,166,171,180,192,205,208,211,214,219,237,244,245,249,252,254,267,293,305,309,311,327,328,333,341,342,344,354,357,368,392,439,449,450,456,469,479,488,491,500,501,508,510,514,516,525,526,532,537,538,541,557,563,568,570,573,575,576,577,581,584,585,597,601,612,616,632,633,634,640,644,668,680,682,695,708,710,712,733,736,742,746,751,752,767,775,808,814,882,889,897,899,900,908,914,915,916,922,924,926,936,941,947,955,958,961,965,966,967,969,971,973,976,981,992,1003,1008,1009,1014,1017,1024,1032,1034,1058,1059,1081,1090,1092,1102,1107,1110,1115,1118,1125,1127,1131,1141,1147,1148,1152,1153,1159,1162,1169])

In [ ]:
iefimerida_grel.loc[[0,1,9,17,20,21,25,28,29,30,35,38,45,46,63,68,69,77,78,80,82,83,93,94,110,113,116,123,134,152,158,172,182,202,207,209,224,251,258,285,295,296,316,320,325,343,361,362,371,383,385,388,389,390,397,400,404,411,414,418,432,437,440,441,443,446,457,460,465,467,477,478,483,497,506,528,545,607,610,629,643,654,674,681,686,689,714,718,744,749,750,756,780,782,817,821,825,836,848,856,859,885,888,896,898,928,943,964,968,986,996,1015,1019,1030,1037,1055,1056,1067,1071,1072,1074,1091,1094,1111,1117,1119,1121,1124,1132,1133,1154,1165,1167], 'κόμματα'] = 'Γενική Αρθρογραφία'

In [ ]:
iefimerida_grel.loc[[4,5,8,10,11,14,18,19,22,23,32,33,34,36,47,50,51,54,55,56,60,61,64,66,70,76,84,85,96,98,103,105,106,107,109,115,117,118,120,124,125,126,128,131,136,145,148,150,151,153,155,156,157,159,160,163,168,169,170,173,174,175,177,178,186,189,194,195,198,200,203,212,213,215,216,217,218,222,225,226,231,233,236,238,242,247,248,250,253,255,256,257,259,262,263,264,265,271,272,273,275,276,278,279,280,281,283,297,298,300,302,304,307,308,310,312,313,319,330,331,332,334,336,339,347,348,350,358,359,360,366,370,372,375,377,379,382,384,386,387,393,394,398,399,402,405,413,416,417,421,423,429,431,433,442,445,451,452,453,454,459,461,462,464,466,468,470,475,476,482,484,485,490,492,494,495,499,504,507,515,519,521,522,524,530,531,535,539,540,543,544,548,550,553,555,558,559,560,562,565,566,574,578,580,582,583,587,588,592,594,598,600,602,604,605,606,608,609,614,620,624,625,626,631,636,637,641,647,649,650,651,659,662,663,666,672,675,676,677,684,687,688,690,692,694,696,698,699,701,703,706,707,709,711,716,722,723,724,727,732,734,735,738,740,741,743,758,761,762,765,771,773,776,777,778,783,787,794,800,804,805,807,810,813,815,818,820,824,826,827,832,833,834,835,837,838,839,842,843,844,845,852,853,854,855,857,864,865,867,869,870,871,876,877,881,883,887,890,895,901,902,903,909,912,918,919,920,930,932,933,934,935,937,938,940,944,945,948,951,952,957,959,970,972,974,975,980,982,987,989,990,991,994,995,1001,1004,1006,1020,1022,1026,1028,1036,1040,1041,1045,1048,1057,1060,1063,1065,1073,1078,1084,1085,1086,1087,1089,1095,1097,1098,1100,1106,1108,1109,1113,1126,1128,1129,1134,1137,1138,1142,1143,1145,1150,1151,1155,1158,1161,1163,1170,1176], 'κόμματα'] = 'ΝΔ'

In [ ]:
iefimerida_grel.loc[[7,15,26,42,43,52,58,59,62,67,75,81,87,89,91,97,99,101,104,108,111,112,119,127,129,130,132,137,137,144,146,154,164,183,184,185,187,188,190,193,196,197,204,210,220,221,223,227,229,234,240,241,243,261,266,268,268,269,277,294,299,301,306,315,321,324,337,345,349,351,352,354,356,369,374,380,381,395,400,403,408,409,419,424,436,444,456,463,471,473,480,481,487,489,493,496,502,505,509,512,513,520,523,527,529,533,536,546,549,551,552,556,564,572,590,595,599,603,611,618,619,621,630,639,642,648,653,657,658,660,664,671,678,679,685,691,693,697,702,713,719,725,728,731,745,747,763,769,772,779,781,791,793,796,799,802,806,811,823,830,840,849,851,858,860,866,872,878,879,886,891,892,894,910,911,913,923,925,927,939,950,954,978,979,985,997,998,1007,1010,1011,1027,1029,1035,1047,1049,1052,1054,1061,1075,1088,1093,1099,1101,1103,1105,1122,1136,1139,1164,1166,1168,1171], 'κόμματα'] = 'ΣΥΡΙΖΑ'

In [ ]:
iefimerida_grel.loc[[2,13,24,31,39,48,49,65,74,90,95,114,140,147,161,167,176,181,201,206,232,235,239,270,274,282,286,287,289,290,318,323,326,335,367,378,391,396,415,427,474,486,503,511,517,534,542,554,561,569,579,586,591,596,615,623,628,638,656,665,670,673,683,700,705,717,730,739,754,766,774,798,812,822,828,831,846,861,863,873,875,880,884,893,905,929,946,949,983,988,1018,1023,1025,1031,1038,1039,1051,1064,1076,1083,1096,1104,1114,1116,1118,1120,1144,1149,1157,1172,1174], 'κόμματα'] = 'ΠΑΣΟΚ'

In [ ]:
iefimerida_grel.loc[[3,6,16,27,37,44,79,102,139,179,191,199,228,246,314,317,329,338,355,406,410,422,425,430,434,447,472,547,567,589,617,627,645,646,667,704,720,721,757,785,819,829,862,907,942,956,984,1042,1069,1079], 'κόμματα'] = 'ΚΚΕ'

In [ ]:
iefimerida_grel.loc[[322,364,373,412,420,428,435,448,655,763,904,1002,1005,1043,1062,1123,1135,1160,1175,1177], 'κόμματα'] = 'Ελληνική Λύση'

In [ ]:
iefimerida_grel.loc[[12,88,143,165,230,260,284,291,292,303,340,365,376,407,426,438,498,518,571,613,635,652,661,768,784,786,792,797,803,809,874,906,963,977,1033,1044,1050,1053,1070,1077,1080,1082,1112,1130,1140,1146,1156], 'κόμματα'] = 'ΜέΡΑ25'

In [ ]:
iefimerida_grel.loc[[135,458,593,622,715,726,737,748,753,755,759,760,770,788,790,795,801,816,841,847,850,868,917,921,931,953,960,962,993,998,1000,1012,1013,1016,1046,1066,1068,1173], 'κόμματα'] = 'Ακροδεξιά'

In [ ]:
iefimerida_grel.loc[[669,1021], 'κόμματα'] = 'Εξωκοινοβουλευτική Αριστερά'

In [ ]:
iefimerida_grel_Σχόλιο =  iefimerida_grel.loc[[1,9,39,43,46,56,58,59,60,62,69,71,73,87,88,90,94,97,99,101,119,135,146,152,164,176,183,186,188,190,201,202,207,223,224,225,226,227,277,284,291,292,315,320,345,360,345,360,362,371,379,381,385,388,390,394,395,396,400,402,403,406,415,417,424,465,473,482,489,490,493,495,502,505,507,512,513,544,545,549,550,551,556,578,610,615,618,622,635,639,643,656,659,658,660,661,663,667,671,678,685,693,702,705,706,728,744,747,748,753,763,765,770]]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nlp = spacy.load('el_core_news_md')

In [ ]:
nlp.max_length = 10000000

In [ ]:
iefimerida_full_text = iefimerida_grel['text'].str.cat(sep = ' ')

In [ ]:
iefimerida_full_doc = nlp(iefimerida_full_text)

In [ ]:
from joblib import dump, load

In [ ]:
#dump(iefimerida_full_doc , 'iefimerida_full_doc.joblib')

In [ ]:
iefimerida_full_doc = load('iefimerida_full_doc.joblib')

In [ ]:
lemmatized_text = ' '.join(token.lemma_ for token in iefimerida_full_doc)

In [ ]:
stopwords = nlp.Defaults.stop_words
stopwords.add("ς")
stopwords.add("μπορώ")
stopwords.add("αναφέρω")
stopwords.add("υπάρχω")
stopwords.add("γίνομαι")
stopwords.add("ή")
stopwords.add("κάνω")
stopwords.add("θέλω")
stopwords.add("κ")
stopwords.add("λέγω")

In [ ]:
lemmatized_text = lemmatized_text.replace("ΣΥΡΙΖΑς", "ΣΥΡΙΖΑ")
wordcloud_iefimerida_grel = WordCloud(
    stopwords = nlp.Defaults.stop_words,
    width = 2000,
    height = 1000,
    background_color = 'black'
 ).generate(lemmatized_text)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud_iefimerida_grel, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [ ]:
count_vector = cv.fit_transform(iefimerida_grel["text"])

In [ ]:
iefimerida_grel_results = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=40, ngram_range=(2,2))
count_vector = cv.fit_transform(iefimerida_grel["text"])
iefimerida_grel_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
iefimerida_grel_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = { "κυριάκος μητσοτάκης":           717,
"νέα δημοκρατία" :               493,
"νέας δημοκρατίας" :             414,
"21ης μαΐου"     :               346,
"αλέξης τσίπρας" :               329,
"αλέξη τσίπρα"   :               271,
"τέσσερα χρόνια"  :              251,
"πρόεδρος συριζα" :              233,
"πασοκ κιναλ"  :                 178,
"φίλες φίλοι"    :               171,
"πρόεδρος πασοκ" :               164,
"συριζα πς"    :                 159,
"Νίκος Ανδρουλάκης"  :           158,
"κυριάκου μητσοτάκη" :           147,
"συριζα πασοκ" :                 146,
"δισ ευρώ"  :                    144,
"κυριάκο μητσοτάκη" :            142,
"ελληνικός λαός" :               137,
"κυβέρνηση ηττημένων" :           137,
"Ανδρέα Παπανδρέου"  :           131,
"21 μαΐου" :                     129,
"απλή αναλογική" :               118,
"νδ συριζα" :                    116,
"αρείου πάγου" :                  112,
"θέσεις εργασίας" :               107,
"κυβέρνηση συνεργασίας" :         106,
"ειδικού σκοπού" :               103,
"Ελληνική Λύση" :                101,
"αξιωματικής αντιπολίτευσης" :     99,
"κυβερνητικός εκπρόσωπος" :       96,
"Δημήτρης Κουτσούμπας" :          96,
"πρόγραμμα συριζα" :               94,
"ελληνικού λαού" :                94,
"επενδυτική βαθμίδα" :            93,
"ταμείο ανάκαμψης" :              93
}
data["Νέα Δημοκρατία/Νέας Δημοκρατίας"] = data.pop("νέας δημοκρατίας") + data.pop("νέα δημοκρατία")
data['Κυριάκος Μητσοτάκης/Κυριάκου Μητσοτάκη/Κυριάκο Μητσοτάκη'] = data.pop('κυριάκος μητσοτάκης')  + data.pop('κυριάκου μητσοτάκη') + data.pop('κυριάκο μητσοτάκη')
data['Αλέξης Τσίπρας/Αλέξη Τσίπρα'] = data.pop('αλέξης τσίπρας') + data.pop('αλέξη τσίπρα')
df = pd.DataFrame(list(data.items()), columns=['Διγράμματα', 'Αναφορές'])

df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(df['Διγράμματα'], df['Αναφορές'], color='black', edgecolor= 'black')
bars = plt.barh(df['Διγράμματα'], df['Αναφορές'], color='black', edgecolor= 'black')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='red',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα')
plt.xlim(0,1200)
plt.title('Διγράμματα Εθνικών Εκλογών 2023 - Iefimerida')
plt.gca().invert_yaxis() 
plt.show()


In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=25, ngram_range=(3,5))
count_vector = cv.fit_transform(iefimerida_grel["text"])
iefimerida_grel_polygrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
iefimerida_grel_polygrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = { "εκλογές 21ης μαΐου"   :               132,
"πασοκ κινήματος αλλαγής"    :          90,
"πρωθυπουργός κυριάκος μητσοτάκης" :    83,
"ΜέΡΑ25 συμμαχία ρήξη"   :              80,
"Εθνικό Σύστημα Υγείας"   :             69,
"κυβέρνηση Νέας Δημοκρατίας" :          66,
"κυβέρνηση ειδικού σκοπού" :            62,
"κάλπη 21ης μαΐου"  :                   60,
"πρόεδρος συριζα πς" :                  58,
"συριζα προοδευτική συμμαχία"   :       58,
"κεντρικό δελτίο ειδήσεων"  :           52,
"πρόεδρος πασοκ κιναλ"   :              44,
"πρωθυπουργό κυριάκο μητσοτάκη" :        43,
"εκλογών 21ης μαΐου"  :                 42,
"πρόεδρος πασοκ κινήματος αλλαγής" :    39,
"κάλπες 21ης μαΐου"  :                  37,
"νέες θέσεις εργασίας"  :               36,
"νδ συριζα πασοκ" :                     36,
"συριζα αλέξης τσίπρας"  :              35,
"κεντρικής επιτροπής KKE Δημήτρης Κουτσούμπας" :              34,
"σταθερά τολμηρά μπροστά" :            33,
"αυτοδύναμη Nέα Δημοκρατία"  :          31,
"κυβερνητικός εκπρόσωπος Άκης Σκέρτσος" :        30}

data["πρόεδρος ΠΑΣΟΚ κινήματος αλλαγής"] = data.pop("πρόεδρος πασοκ κιναλ") + data.pop("πρόεδρος πασοκ κινήματος αλλαγής")
data["πρωθυπουργός Κυριάκος Μητσοτάκης"] = data.pop("πρωθυπουργός κυριάκος μητσοτάκης") + data.pop("πρωθυπουργό κυριάκο μητσοτάκη")
data["κάλπη 21ης μαΐου"] = data.pop("κάλπη 21ης μαΐου") + data.pop ("κάλπες 21ης μαΐου")
df = pd.DataFrame(list(data.items()), columns=['Διγράμματα', 'Αναφορές'])

df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(df['Διγράμματα'], df['Αναφορές'], color='black', edgecolor= 'black')
bars = plt.barh(df['Διγράμματα'], df['Αναφορές'], color='black', edgecolor= 'black')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='red',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα')
plt.xlim(0,200)
plt.title('Πολυγράμματα Εθνικών Εκλογών 2023 - Iefimerida')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
iefimerida_Γενική_Αρθρογραφία = iefimerida_grel.loc[iefimerida_grel['κόμματα'] == 'Γενική Αρθρογραφία']
iefimerida_ΝΔ = iefimerida_grel.loc[iefimerida_grel['κόμματα'] == 'ΝΔ']
iefimerida_ΣΥΡΙΖΑ = iefimerida_grel.loc[iefimerida_grel['κόμματα'] == 'ΣΥΡΙΖΑ']
iefimerida_ΠΑΣΟΚ = iefimerida_grel.loc[iefimerida_grel['κόμματα'] == 'ΠΑΣΟΚ']
iefimerida_ΚΚΕ = iefimerida_grel.loc[iefimerida_grel['κόμματα'] == 'ΚΚΕ']
iefimerida_ΜέΡΑ25 = iefimerida_grel.loc[iefimerida_grel['κόμματα'] == 'ΜέΡΑ25']
iefimerida_Ακροδεξιά = iefimerida_grel.loc[iefimerida_grel['κόμματα'] == 'Ακροδεξιά']
iefimerida_Ελληνική_Λύση = iefimerida_grel.loc[iefimerida_grel['κόμματα'] == 'Ελληνική Λύση']
iefimerida_Εξωκοινοβουλευτική_Αριστερά = iefimerida_grel.loc[iefimerida_grel['κόμματα'] == 'Εξωκοινοβουλευτική Αριστερά']

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=40, ngram_range=(2,2))
count_vector = cv.fit_transform(iefimerida_ΝΔ["text"])
iefimerida_grel_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
iefimerida_ΝΔ_full_text = iefimerida_ΝΔ['text'].str.cat(sep = ' ')

In [ ]:
iefimerida_ΝΔ_full_doc = nlp(iefimerida_ΝΔ_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=30, ngram_range=(3,5))
count_vector = cv.fit_transform(iefimerida_ΝΔ["text"])
iefimerida_ΝΔ_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
iefimerida_ΝΔ_bigrams.sum(axis =0).sort_values(ascending = True)

In [ ]:
data = {"κυριάκος μητσοτάκης":        620,
"νέα δημοκρατία"  :           337,
"νέας δημοκρατίας" :          290,
"τέσσερα χρόνια"  :           180,
"21ης μαΐου" :                174,
"φίλες φίλοι":                161,
"δισ ευρώ"   :                114,
"κυριάκου μητσοτάκη"  :       113,
"θέσεις εργασίας" :            95,
"κυβέρνηση ηττημένων" :        83,
"επενδυτική βαθμίδα":         81,
"νέα παιδιά"      :            79,
"σύστημα υγείας" :             75,
"Aλέξη Tσίπρα"  :              74,
"κυβερνητικός εκπρόσωπος":     67,
"ταμείο ανάκαμψης" :           65,
"εθνικό σύστημα υγείας"  :            62,
"κυριάκο μητσοτάκη" :          61,
"21η μαΐου" :                  61,
"προσωπική διαφορά" :          58,
"Αντώνης Σρόιτερ" :            58,
"Κατερίνα Καινούργιου" :       55,
"απλή αναλογική"  :            54,
"πρωτογενή τομέα" :            53}

data["Νέα Δημοκρατία/Νέας Δημοκρατίας"] = data.pop("νέας δημοκρατίας") + data.pop("νέα δημοκρατία")
data["Κυριάκος Μητσοτάκης/Κυριάκου Μητσοτάκη"] = data.pop("κυριάκος μητσοτάκης") + data.pop("κυριάκου μητσοτάκη")

df = pd.DataFrame(list(data.items()), columns=['Διγράμματα', 'Αναφορές'])
df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(df['Διγράμματα'], df['Αναφορές'], color='blue', edgecolor= 'black')
bars = plt.barh(df['Διγράμματα'], df['Αναφορές'], color='blue', edgecolor= 'black')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='black',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα')
plt.xlim(0,1000)
plt.title('Διγράμματα Εθνικών Εκλογών 2023- Νέα Δημοκρατία - Iefimerida')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
iefimerida_ΣΥΡΙΖΑ_full_text = iefimerida_ΣΥΡΙΖΑ['text'].str.cat(sep = ' ')

In [ ]:
iefimerida_ΣΥΡΙΖΑ_full_doc = nlp(iefimerida_ΣΥΡΙΖΑ_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=40, ngram_range=(2,5))
count_vector = cv.fit_transform(iefimerida_ΣΥΡΙΖΑ["text"])
iefimerida_ΣΥΡΙΖΑ_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
iefimerida_ΣΥΡΙΖΑ_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = {"Αλέξης Τσίπρας" :             183,
"πρόεδρος ΣΥΡΙΖΑ" :             167,
"συριζα πς"  :                   134,
"αλέξη τσίπρα" :                  85,
"μεσαία τάξη" :                    56,
"τέσσερα χρόνια" :                50,
"συριζα προοδευτική συμμαχία" : 49,
"νίκη ΣΥΡΙΖΑ"    :                46,
"Πόπη Τσαπανίδου"   :             44,
"προοδευτική κυβέρνηση"  :        40,
"Κυριάκο Μητσοτάκη" :             40,
"κυβέρνηση Μητσοτάκη" :           39,
"πρόγραμμα ΣΥΡΙΖΑ" :             39,
"συμβόλαιο αλλαγής"  :            34,
"ελληνικός λαός"  :               33,
"21ης Μαΐου"  :                   32,
"αξιωματικής αντιπολίτευσης" :    30,
"απλή αναλογική"   :              28,
"προοδευτικής συνεργασίας" :       28,
"εθνικός λόγος" :                 27,
"Τσίπρας είπε" :                  27,
"Γιάνη Βαρουφάκη" :               26,
"αύξηση μισθών" :                 23,
"νέας δημοκρατίας" :              23,
"νέα δημοκρατία" :                23,
"Ανδρέα Παπανδρέου" :             23,
"εκπρόσωπος τύπου ΣΥΡΙΖΑ" :             23,
"μείωση τιμών"  :                23,
"Χρυσή Αυγή"  :                   22,
"πρώτης κατοικίας" :             22,
"κυβέρνηση προοδευτικής συνεργασίας"  :       22,
}

data["ΣΥΡΙΖΑ ΠΣ/ ΣΥΡΙΖΑ Προοδευτική Συμμαχία"] = data.pop("συριζα πς") + data.pop("συριζα προοδευτική συμμαχία")
data["Αλέξης Τσίπρας/Αλέξη Τσίπρα"] = data.pop("Αλέξης Τσίπρας") + data.pop("αλέξη τσίπρα")
data["Νέα Δημοκρατία/Νέας Δημοκρατίας"] = data.pop("νέας δημοκρατίας") + data.pop("νέα δημοκρατία")
df = pd.DataFrame(list(data.items()), columns=['Διγράμματα', 'Αναφορές'])
df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(df['Διγράμματα'], df['Αναφορές'], color='mediumpurple', edgecolor= 'black')
bars = plt.barh(df['Διγράμματα'], df['Αναφορές'], color= 'mediumpurple', edgecolor= 'black')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='black',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα/Πολυγράμματα')
plt.xlim(0,300)
plt.title('Διγράμματα/Πολυγράμματα Εθνικών Εκλογών 2023- ΣΥΡΙΖΑ ΠΣ - Iefimerida')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
iefimerida_ΠΑΣΟΚ_full_text = iefimerida_ΠΑΣΟΚ['text'].str.cat(sep = ' ')

In [ ]:
iefimerida_ΠΑΣΟΚ_full_doc = nlp(iefimerida_ΠΑΣΟΚ_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=30, ngram_range=(2,5))
count_vector = cv.fit_transform(iefimerida_ΠΑΣΟΚ["text"])
iefimerida_ΠΑΣΟΚ_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out())

In [ ]:
iefimerida_ΠΑΣΟΚ_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = {"πρόεδρος ΠΑΣΟΚ"  :                    122,
"Νίκος Ανδρουλάκης" :                  103,
"ΠΑΣΟΚ ΚΙΝΑΛ"           :              79,
"ΠΑΣΟΚ Κινήματος Αλλαγής" :             73,
"Νέα Δημοκρατία"  :                     53,
"Ανδρέα Παπανδρέου" :                   46,
"Νέας Δημοκρατίας" :                    42,
"πρόεδρος ΠΑΣΟΚ Κινήματος Αλλαγής" :    32,
"21 Μαΐου"  :                           32,
"Αλέξη Τσίπρα"  :                       32,
"ελληνικό λαό"   :                      30,
"Πρόεδρος ΠΑΣΟΚ ΚΙΝΑΛ" :                30,
"ελληνικού λαού"  :                     30,
"Νίκου Ανδρουλάκη" :                    29,
"Ανδρέας Παπανδρέου"     :                    25,
"νέα γενιά"         :                  23
}

data["Ανδρέας Παπανδρέου/Ανδρέα Παπανδρέου "] = data.pop("Ανδρέας Παπανδρέου") + data.pop("Ανδρέα Παπανδρέου")
data["Νέας Δημοκρατίας/Νέα Δημοκρατία"] = data.pop("Νέα Δημοκρατία") + data.pop("Νέας Δημοκρατίας")
data["Νίκος Ανδρουλάκης/Νίκου Ανδρουλάκη"] = data.pop("Νίκος Ανδρουλάκης") + data.pop("Νίκου Ανδρουλάκη")
df = pd.DataFrame(list(data.items()), columns=['Διγράμματα', 'Αναφορές'])
df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(df['Διγράμματα'], df['Αναφορές'], color='green', edgecolor= 'black')
bars = plt.barh(df['Διγράμματα'], df['Αναφορές'], color= 'green', edgecolor= 'black')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='black',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα/Πολυγράμματα')
plt.xlim(0,200)
plt.title('Διγράμματα/Πολυγράμματα Εθνικών Εκλογών 2023- ΠΑΣΟΚ-ΚΙΝΑΛ - Iefimerida')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
iefimerida_ΚΚΕ_full_text = iefimerida_ΚΚΕ['text'].str.cat(sep = ' ')

In [ ]:
iefimerida_ΚΚΕ_full_doc = nlp(iefimerida_ΚΚΕ_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=30, ngram_range=(2,5))
count_vector = cv.fit_transform(iefimerida_ΚΚΕ["text"])
iefimerida_ΚΚΕ_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out())

In [ ]:
iefimerida_ΚΚΕ_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = {"δημήτρης κουτσούμπας"  :                59,
"ΝΔ ΣΥΡΙΖΑ" :                                    45,
"ΣΥΡΙΖΑ ΠΑΣΟΚ" :                                 35,
"κεντρικής επιτροπής"  :                         35,
"ΝΔ ΣΥΡΙΖΑ ΠΑΣΟΚ" :                              30,
"επιτροπής ΚΚΕ" :                                30,
"γενικός γραμματέας κεντρικής επιτροπής ΚΚΕ" :    30,
"κε ΚΚΕ" :                                        21,
"δυνατό ΚΚΕ" :                                   20,
"γενικός γραμματέας κεντρικής επιτροπής ΚΚΕ" :    20,
"αντιλαϊκή κυβέρνηση" :                          19,
"γγ ΚΕ" :                                        17,
"ψήφος ΚΚΕ"  :                                   16,
"ΝΔ ΠΑΣΟΚ" :                                      15,
"δημήτρη κουτσούμπα" :                           14,
"δυνατός λαός" :                                 13
}

data["Δημήτρης Κουτσούμπας/Δημήτρη Κουτσούμπα "] = data.pop("δημήτρης κουτσούμπας") + data.pop("δημήτρη κουτσούμπα")
df = pd.DataFrame(list(data.items()), columns=['Διγράμματα/Πολυγράμματα', 'Αναφορές'])
df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(df['Διγράμματα/Πολυγράμματα'], df['Αναφορές'], color='red', edgecolor= 'black')
bars = plt.barh(df['Διγράμματα/Πολυγράμματα'], df['Αναφορές'], color= 'red', edgecolor= 'black')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='black',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα/Πολυγράμματα')
plt.xlim(0,100)
plt.title('Διγράμματα/Πολυγράμματα Εθνικών Εκλογών 2023- ΚΚΕ - Iefimerida')
plt.gca().invert_yaxis() 
plt.show()



In [ ]:
iefimerida_ΜέΡΑ25_full_text = iefimerida_ΜέΡΑ25['text'].str.cat(sep = ' ')

In [ ]:
iefimerida_ΜέΡΑ25_full_doc = nlp(iefimerida_ΜέΡΑ25_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=30, ngram_range=(2,5))
count_vector = cv.fit_transform(iefimerida_ΜέΡΑ25["text"])
iefimerida_ΜέΡΑ25_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out())

In [ ]:
iefimerida_ΜέΡΑ25_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:


data = {                
"ΜέΡΑ25 συμμαχία ρήξη" :                   80,
"σχέδιο Δήμητρα" :                    34,
"γιάνης βαρουφάκης" :                 26,
"επικεφαλής ΜέΡΑ25" :                 22,
"ΣΥΡΙΖΑ ΠΑΣΟΚ"  :                     21,
"γιάνη βαρουφάκη"  :                17,
"επικεφαλής ΜέΡΑ25 συμμαχία ρήξη" :  16,
"ΜέΡΑ25" :                           15,
"χρηματιστήριο ενέργειας"  :          14,
"κλείσουν τράπεζες" :                 13,
"σύστημα Δήμητρα" :                    13,
"κόκκινα δάνεια"  :                   13,
"απλή αναλογική" :                    12
}

data["Γιάνης Βαρουφάκης/Γιάνη Βαρουφάκη"] = data.pop("γιάνης βαρουφάκης") + data.pop("γιάνη βαρουφάκη")
df = pd.DataFrame(list(data.items()), columns=['Διγράμματα/Πολυγράμματα', 'Αναφορές'])
df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(df['Διγράμματα/Πολυγράμματα'], df['Αναφορές'], color='orange', edgecolor= 'black')
bars = plt.barh(df['Διγράμματα/Πολυγράμματα'], df['Αναφορές'], color= 'orange', edgecolor= 'black')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='black',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα/Πολυγράμματα')
plt.xlim(0,100)
plt.title('Διγράμματα/Πολυγράμματα Εθνικών Εκλογών 2023- ΜέΡΑ25 - Iefimerida')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
filepath = "https://raw.githubusercontent.com/datajour-gr/DataJournalism/main/Bachelor%20Lessons%202023/Lesson%2010/NRC_GREEK_Translated_6_2020.csv"

In [ ]:
emolex_df = pd.read_csv(filepath)

In [ ]:
emolex_df = emolex_df.drop_duplicates(subset=['word'])
emolex_df = emolex_df.dropna()
emolex_df.reset_index(inplace = True, drop=True)

In [ ]:
vec = CountVectorizer(analyzer = 'word', vocabulary = emolex_df.word,
                      lowercase=False, 
                      strip_accents = 'unicode',  
                      stop_words= list(nlp.Defaults.stop_words),
                      ngram_range=(1, 2))

In [ ]:
iefimerida_ΠΑΣΟΚ['pos/neg_text'].median()

In [ ]:
iefimerida_ΚΚΕ['pos/neg_text'].median()

In [ ]:
iefimerida_ΜέΡΑ25['pos/neg_text'].median()

In [ ]:
iefimerida_Ακροδεξιά['pos/neg_text'].mean()

In [ ]:
iefimerida_grel['pos/neg_title'].mean()

In [ ]:
iefimerida_ΝΔ['pos/neg_title'].mean()

In [ ]:
iefimerida_ΣΥΡΙΖΑ['pos/neg_title'].mean()

In [ ]:
iefimerida_ΠΑΣΟΚ['pos/neg_title'].mean()

In [ ]:
iefimerida_ΚΚΕ['pos/neg_title'].mean().round(2)

In [ ]:
iefimerida_ΜέΡΑ25['pos/neg_title'].mean().round(2)

In [ ]:
iefimerida_Ακροδεξιά['pos/neg_title'].mean()

In [ ]:
iefimerida_grel.shape


In [ ]:
iefimerida_Γενική_Αρθρογραφία.shape


In [ ]:
iefimerida_ΝΔ.shape


In [ ]:
iefimerida_ΣΥΡΙΖΑ.shape


In [ ]:
iefimerida_ΠΑΣΟΚ.shape


In [ ]:
iefimerida_ΚΚΕ.shape


In [ ]:
iefimerida_ΜέΡΑ25.shape


In [ ]:
iefimerida_Ακροδεξιά.shape 


In [ ]:
iefimerida_Ελληνική_Λύση.shape


In [ ]:
iefimerida_Εξωκοινοβουλευτική_Αριστερά.shape

In [ ]:
from pywaffle import Waffle

parties = ['Νέα Δημοκρατία', 'ΣΥΡΙΖΑ', 'ΠΑΣΟΚ', 'ΚΚΕ', 'ΜέΡΑ25', 'Ακροδεξιά', 'Ελληνική Λύση', 'Γενική Αρθρογραφία', 'Εξωκοινοβουλευτική Αριστερά']
post_counts = [402, 206, 110, 50, 47, 38, 20, 132, 2]

total_posts = sum(post_counts)
percentages = [count / total_posts * 100 for count in post_counts]

colors = ['blue', 'mediumpurple', 'green', 'red', 'darkorange', 'darkseagreen', 'lightsteelblue', 'grey', 'fuchsia']

data = {f"{party} ({count}, {percentage:.1f}%)": count for party, count, percentage in zip(parties, post_counts, percentages)}

fig = plt.figure(
    FigureClass=Waffle, 
    rows=25,  # Adjust the number of rows to change the size of squares
    values=data, 
    colors=colors, 
    legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1), 'fontsize': 10, 'title': 'Κόμματα'},
    title={'label': 'Σύνολο και Ποσοστά δημοσιευμάτων για τα κόμματα - Iefimerida', 'loc': 'center', 'fontsize': 16},
    block_arranging_style='snake',  # Arranges the blocks in a snake style
    figsize=(14, 12)
)

plt.show()


In [ ]:
means = {
    'ΝΔ': {
        'Πολικότητα Κειμένων': iefimerida_ΝΔ['pos/neg_text'].median().round(2),
        'Πολικότητα Τίτλων': iefimerida_ΝΔ['pos/neg_title'].median().round(2)
    },
    'ΣΥΡΙΖΑ': {
        'Πολικότητα Κειμένων': iefimerida_ΣΥΡΙΖΑ['pos/neg_text'].median().round(2),
        'Πολικότητα Τίτλων': iefimerida_ΣΥΡΙΖΑ['pos/neg_title'].median().round(2)
    },
    'ΠΑΣΟΚ': {
        'Πολικότητα Κειμένων': iefimerida_ΠΑΣΟΚ['pos/neg_text'].median().round(2),
        'Πολικότητα Τίτλων': iefimerida_ΠΑΣΟΚ['pos/neg_title'].median().round(2)
    },
    'ΚΚΕ': {
        'Πολικότητα Κειμένων': iefimerida_ΚΚΕ['pos/neg_text'].median().round(2),
        'Πολικότητα Τίτλων': iefimerida_ΚΚΕ['pos/neg_title'].median().round(2)
    },
    'ΜέΡΑ25': {
        'Πολικότητα Κειμένων': iefimerida_ΜέΡΑ25['pos/neg_text'].median().round(2),
        'Πολικότητα Τίτλων': iefimerida_ΜέΡΑ25['pos/neg_title'].median().round(2)
    },
    'Ακροδεξιά': {
        'Πολικότητα Κειμένων': iefimerida_Ακροδεξιά['pos/neg_text'].median().round(2),
        'Πολικότητα Τίτλων': iefimerida_Ακροδεξιά['pos/neg_title'].median().round(2)
    },
    'Ελληνική Λύση': {
        'Πολικότητα Κειμένων': iefimerida_Ελληνική_Λύση['pos/neg_text'].median().round(2),
        'Πολικότητα Τίτλων': iefimerida_Ελληνική_Λύση['pos/neg_title'].median().round(2)
    }
}


means_df = pd.DataFrame(means).T 


color_map = {
    'ΝΔ': 'blue',
    'ΣΥΡΙΖΑ': 'mediumpurple',
    'ΠΑΣΟΚ': 'green',
    'ΚΚΕ': 'red',
    'ΜέΡΑ25': 'darkorange',
    'Ακροδεξιά': 'darkseagreen',
    'Ελληνική Λύση': 'lightblue'
}

new_col_titles = ['Πολικότητα Κειμένων', 'Πολικότητα Τίτλων']


fig, ax = plt.subplots(figsize=(14, 8))

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.set_frame_on(False)

table = ax.table(cellText=means_df.values,
                 rowLabels=means_df.index,
                 colLabels=new_col_titles,
                 cellLoc='center',
                 loc='center',
                 colColours=['lightgrey'] * len(new_col_titles),
                 rowColours=['white'] * len(means_df.index))  


for i, label in enumerate(means_df.index):
    row_color = color_map.get(label, 'white')  
    for j in range(len(new_col_titles)):
        table[(i + 1, j)].set_facecolor(row_color)  

table.auto_set_font_size(False)
table.set_fontsize(14)  
table.scale(1.5,3)   


In [ ]:
short_df = iefimerida_grel[["title","Conflict"]]

In [ ]:
short_df


In [ ]:
import re

frames = {
    'Σύγκρουση': [
        'κόντρα', 'VS', 'μάχη', 'επίθεση', 'αντιπαράθεση', '«σφάχτηκαν»', '«επιτίθεται»', "ή", "δίλημμα",
        '«πυρά»', '«Πυρά»','πυρά', 'Πυρά', 'μπηχτή', 'μεταξύ', 'σύγκρουση', '«σκοτωμός»', 'απαντά', '«σπόντα»',
        '«μάχης»', 'μια πλευρά', 'Μητσοτάκη-Τσίπρα', 'απάντηση', 'καυγά', 'αίμα', 'απαντάμε',
        'διαφορά', 'έναντι', 'σύγκριση', 'κατά', 'εναντίον','προβάδισμα', 'Μητσοτάκης-Τσίπρας', 'ΝΔ-ΣΥΡΙΖΑ' 'ΝΔ - ΣΥΡΙΖΑ',
    ]
}

def detect_conflict_frames(title, frames):
    conflict_keywords = frames['Σύγκρουση']
    
    pattern = r'\b(?:' + '|'.join(map(re.escape, conflict_keywords)) + r')\b'

    return bool(re.search(pattern, title))


iefimerida_grel['Conflict'] = iefimerida_grel['title'].apply(lambda x: 1 if detect_conflict_frames(x, frames) else 0)




In [ ]:
iefimerida_grel['Conflict'].value_counts()

In [ ]:
iefimerida_grel.shape

In [ ]:
11.62